In [1]:
import contextily as ctx
import folium
import geopandas as gpd
import pandas as pd
import requests
import matplotlib.pyplot as plt
from mapclassify import classify
from matplotlib import colormaps as cm

from folium.plugins import GroupedLayerControl


# Préparation des données géographiques des quartiers

In [2]:
# Importation des données géographiques des quartiers de Marseille
url = 'https://www.data.gouv.fr/api/1/datasets/r/8a8f7f54-7f91-482c-a78c-dd09d893d1b6'
file = requests.get(url)
data = file.content
quartiers_data = gpd.read_file(data)

In [3]:
quartiers_data.info()
quartiers_data.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   DEPCO     111 non-null    object  
 1   NOM_CO    111 non-null    object  
 2   NOM_QUA   111 non-null    object  
 3   geometry  111 non-null    geometry
dtypes: geometry(1), object(3)
memory usage: 3.6+ KB


,DEPCO,NOM_CO,NOM_QUA,geometry
0,13201,Marseille 1er Arrondissemen,BELSUNCE,"MULTIPOLYGON (((5.38086 43.29924, 5.38087 43.2..."
1,13201,Marseille 1er Arrondissemen,CHAPITRE,"MULTIPOLYGON (((5.38525 43.29906, 5.38485 43.2..."
2,13201,Marseille 1er Arrondissemen,NOAILLES,"MULTIPOLYGON (((5.3816 43.29573, 5.38177 43.29..."
3,13201,Marseille 1er Arrondissemen,OPERA,"MULTIPOLYGON (((5.37729 43.29222, 5.37663 43.2..."
4,13201,Marseille 1er Arrondissemen,SAINT CHARLES,"MULTIPOLYGON (((5.38022 43.30141, 5.38007 43.3..."


In [4]:
quartiers_data.loc[quartiers_data['NOM_QUA'] == 'ENDOUME']

,DEPCO,NOM_CO,NOM_QUA,geometry
29,13207,Marseille 7e Arrondissemen,ENDOUME,"MULTIPOLYGON (((5.34923 43.28452, 5.34919 43.2..."
30,13207,Marseille 7e Arrondissemen,ENDOUME,"MULTIPOLYGON (((5.34756 43.29063, 5.3489 43.28..."


In [5]:
# l'observation 30 correspond aux îles, on la supprime car outlier sur plusieurs types de flux
quartiers_data = quartiers_data.drop([30])

# Préparation des données sur les pav

In [6]:
# Importation des données concernant les points d'apport volontaire
url = 'https://www.data.gouv.fr/api/1/datasets/r/e46c6879-49e7-4727-8f3b-62df90ac5a5a'
gouv_data = pd.read_csv(url, sep = ';')
gouv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12758 entries, 0 to 12757
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   OBJECTID                          12758 non-null  int64  
 1   Ex identifiant CT                 12049 non-null  object 
 2   Date création mobilier            12675 non-null  object 
 3   Famille Osis                      12758 non-null  object 
 4   Type de colonne                   12744 non-null  object 
 5   Modèle                            11431 non-null  object 
 6   Volume intérieur m3               8982 non-null   float64
 7   longitude                         12758 non-null  float64
 8   latitude                          12758 non-null  float64
 9   Date de mise à jour de la donnée  12758 non-null  object 
 10  Code Commune INSEE                12723 non-null  float64
 11  Commune                           12723 non-null  object 
 12  Posi

In [7]:
# Extraction des variables d'intérêt
pav_data = gouv_data[['OBJECTID', 'Code Commune INSEE', 'Quartier', 'Volume intérieur m3', 'Type de flux', 'longitude', 'latitude']]

In [8]:
# Extraction des observations concernant les quartiers de Marseille
pav_marseille = pav_data.loc[(pav_data['Code Commune INSEE'] > 13200) & (pav_data['Code Commune INSEE'] < 13217)]
pav_marseille = pav_marseille.dropna(axis = 0, subset = ['Quartier'])
pav_marseille.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5184 entries, 20 to 12699
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OBJECTID             5184 non-null   int64  
 1   Code Commune INSEE   5184 non-null   float64
 2   Quartier             5184 non-null   object 
 3   Volume intérieur m3  3378 non-null   float64
 4   Type de flux         5184 non-null   object 
 5   longitude            5184 non-null   float64
 6   latitude             5184 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 324.0+ KB


# Préparation des données de population

In [9]:
#import des données de population des quartiers de Marseille
df_pop_quartier = pd.read_csv("../data/raw/pop_marseille_2022.csv", sep=";")

# Traitement des dataframes

## Jointure entre les données des quartiers et les données de population

In [10]:
df_pop_quartier['quartier'] = df_pop_quartier['quartier'].apply(lambda x : x.replace('-', ' ').upper())

In [11]:
# On va modifier certains noms de quartiers pour qu'ils correspondent entre les deux DataFrames
mapping = {'OPÉRA' : 'OPERA',
            'HÔTEL DE VILLE' : 'HOTEL DE VILLE',
            'SAINT MAURON': 'SAINT MAURONT',
           'VILETTE': 'LA VILLETTE',
           'VILLETTE' : 'LA VILLETTE',
           'LA VILETTE' : 'LA VILLETTE',
           'BLANCARDE': 'LA BLANCARDE',
           'CHARTREUX': 'LES CHARTREUX',
           'CAMAS': 'LE CAMAS',
            'CONCEPTION': 'LA CONCEPTION',
            'PRÉFECTURE' : 'PREFECTURE',
            'PHARO': 'LE PHARO',
            'PÉRIER' : 'PERIER',
            'ROUET': 'LE ROUET',
            'VIELLE CHAPELLE': 'VIEILLE CHAPELLE',
            'BAUMETTES': 'LES BAUMETTES',
            'REDON': 'LE REDON',
            'VAUFRÈGES' : 'VAUFREGES',
            'CAPELETTE': 'LA CAPELETTE',
            'LA MILLIÈRE' : 'LA MILLIERE',
            'LA FOURRAGÈRE' : 'LA FOURRAGERE',
            'SAINT BARNABÉ' : 'SAINT BARNABE',
            'SAINT JEAN DU DÉSERT' : 'SAINT JEAN DU DESERT',
            'CHATEAU-GOMBERT' : 'CHATEAU GOMBERT',
            'CHÂTEAU GOMBERT' : 'CHATEAU GOMBERT',
            'CROIX ROUGE': 'LA CROIX ROUGE',
            'MALPASSÉ' : 'MALPASSE',
            'LES MÉDECINS' : 'LES MEDECINS',
            'MOURETS': 'LES MOURETS',
            'SAINT JÉRÔME' : 'SAINT JEROME',
            'SAINT BARTHÉLEMY' : 'SAINT BARTHELEMY',
            'BORELS': 'LES BORELS',
            'SAINT ANDRÉ' : 'SAINT ANDRE'
            }

In [12]:
df_pop_quartier['quartier'] = df_pop_quartier['quartier'].replace(mapping)
quartiers_data['NOM_QUA'] = quartiers_data['NOM_QUA'].replace(mapping)

In [13]:
quartiers_merge = df_pop_quartier.merge(quartiers_data, left_on='quartier', right_on = 'NOM_QUA').drop('NOM_QUA', axis = 1)

In [14]:
quartiers_merge = quartiers_merge.rename({'quartier' : 'Quartier'}, axis = 1)
quartiers_merge

,Quartier,population,DEPCO,NOM_CO,geometry
0,BELSUNCE,8825,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38086 43.29924, 5.38087 43.2..."
1,CHAPITRE,6497,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38525 43.29906, 5.38485 43.2..."
2,NOAILLES,4304,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.3816 43.29573, 5.38177 43.29..."
3,OPERA,5468,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.37729 43.29222, 5.37663 43.2..."
4,SAINT CHARLES,8390,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38022 43.30141, 5.38007 43.3..."
...,...,...,...,...,...
105,LA VISTE,6665,13215,Marseille 15e Arrondissemen,"MULTIPOLYGON (((5.35771 43.35337, 5.35756 43.3..."
106,L'ESTAQUE,5807,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.32691 43.36112, 5.32694 43.3..."
107,LES RIAUX,722,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.30862 43.36508, 5.30878 43.3..."
108,SAINT ANDRE,3739,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.33997 43.34367, 5.33992 43.3..."


## Jointure des autres dataframe

In [15]:
# Extraction des observations concernant les quartiers de Marseille
pav_marseille = pav_data.loc[(pav_data['Code Commune INSEE'] > 13200) & (pav_data['Code Commune INSEE'] < 13217)]
pav_quartiers = pav_marseille.dropna(axis = 0, subset = ['Quartier'])
pav_quartiers.head()


,OBJECTID,Code Commune INSEE,Quartier,Volume intérieur m3,Type de flux,longitude,latitude
20,126548,13201.0,SAINT CHARLES,2.2,Verre,5.390392,43.304831
21,126549,13201.0,SAINT CHARLES,3.2,Biflux,5.390395,43.304833
39,126606,13207.0,SAINT VICTOR,3.0,Biflux,5.368802,43.285029
40,126609,13201.0,THIERS,3.0,Verre,5.385471,43.296029
41,126611,13201.0,SAINT CHARLES,3.0,Biflux,5.393332,43.305732


In [16]:
# conversion en geodataframe
pav_quartiers = gpd.GeoDataFrame(
    pav_quartiers,
    geometry=gpd.points_from_xy(
        pav_quartiers['longitude'],
        pav_quartiers['latitude'],
        crs="EPSG:4326"))

pav_quartiers = pav_quartiers.to_crs(3857)

In [17]:
# Uniformisation des noms de quartiers 

mapping = {
        'SAINT MAURON' : 'SAINT MAURONT',
        'VIELLE CHAPELLE' : 'VIEILLE CHAPELLE',
        'POINTE ROUGE' : 'LA POINTE ROUGE',
        'LE ROUCAS' : 'LE ROUCAS BLANC',
        'ROUCAS BLANC' : 'LE ROUCAS BLANC',
        'CHUTES LAVIE' : 'LES CHUTES LAVIE',
        'LES CHUTES LAVIES' : 'LES CHUTES LAVIE',
        'CINQ AVENUES' : 'LES CINQ AVENUES',
        'GRANDS CARMES' : 'LES GRANDS CARMES',
        'ST BARNANE' : 'SAINT BARNABE',
        'STE MARGUERITE' : 'SAINTE MARGUERITE'
    }

quartiers_merge['Quartier'] = quartiers_merge['Quartier'].replace(mapping)
pav_quartiers['Quartier'] = pav_quartiers['Quartier'].replace(mapping)


In [18]:
# Trop de données manquantes sur le volume de points d'apport à l'échelle du quartier...
print(pav_quartiers.isna().mean())


OBJECTID               0.00000
Code Commune INSEE     0.00000
Quartier               0.00000
Volume intérieur m3    0.34838
Type de flux           0.00000
longitude              0.00000
latitude               0.00000
geometry               0.00000
dtype: float64


In [19]:
# On se contente de récupérer le nombre de points d'apport par quartier
nb_pav_quartiers = pav_quartiers.groupby(['Quartier', 'Type de flux']).count()['OBJECTID'].reset_index()
nb_pav_quartiers

,Quartier,Type de flux,OBJECTID
0,ARENC,Biflux,8
1,ARENC,Biodechets,1
2,ARENC,OM,1
3,ARENC,Textile,1
4,ARENC,Verre,8
...,...,...,...
453,VERDURON,Verre,4
454,VIEILLE CHAPELLE,Biflux,13
455,VIEILLE CHAPELLE,Biodechets,9
456,VIEILLE CHAPELLE,Textile,1


In [20]:
types_flux = pav_quartiers['Type de flux'].unique()
types_flux

array(['Verre', 'Biflux', 'OM', 'Carton', 'Biodechets', 'Textile'],
      dtype=object)

In [21]:
# ajout des lignes pour lesquelles la combinaison quartier/types de flux n'existe pas
# afin que ces quartiers apparaissent tout de même sur la carte, avec une densité de 0
for quartier in nb_pav_quartiers['Quartier'].unique():
    quartier_df = nb_pav_quartiers.loc[nb_pav_quartiers['Quartier'] == quartier]
    for type in types_flux:
        if type not in quartier_df['Type de flux'].unique():
            new_line = pd.DataFrame.from_dict(
                {
                    'Quartier' : [quartier],
                    'Type de flux' : [type],
                    'OBJECTID' : [0]
                }
            )
            nb_pav_quartiers = pd.concat([nb_pav_quartiers, new_line])

In [22]:
# merge des données de carte, de population et de volume des points d'apport
marseille = (quartiers_merge
             .merge(nb_pav_quartiers, on = 'Quartier', how = 'right')
             .rename({'OBJECTID' : 'nb_points'}, axis = 1)
)

marseille['nb_points'] = marseille['nb_points'].fillna(0).astype(int)



# On passe en Geodataframe pour les cartes

In [23]:
#Conversion de marseille en geodataframe
gdf_marseille = gpd.GeoDataFrame(marseille, geometry='geometry').to_crs(3857)

In [24]:
# Calcul de la densité des points d'apport par quartier
gdf_marseille['pav_pour_1000_hab'] = gdf_marseille['nb_points']/gdf_marseille['population']*1000

In [25]:
pav_quartiers.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Carte du nombre de point d'apport

In [26]:
dict_pav = {}
dict_quartiers = {}

for type in types_flux:
    dict_pav[type] = pav_quartiers.loc[pav_quartiers['Type de flux'] == type]
    dict_quartiers[type] = gdf_marseille.loc[gdf_marseille['Type de flux'] == type]

In [28]:
# fond de carte
map = folium.Map(
    location = [43.275, 5.4],
    tiles = 'cartodbpositron',
    zoom_start = 12,
    max_zoom=20,
    control_scale=True
    )



# ajout des couches pour chaque type de flux
layers = {}

for type in types_flux:
    layer_name = 'fg_'+ type 
    label = type
    layers[layer_name] = folium.FeatureGroup(name = label).add_to(map)

    
    dict_pav[type].explore(
        m = layers[layer_name],
        marker_kwds=dict(radius=1, fill=True),
        tooltip = ['OBJECTID', 'Quartier'],
        color = 'Red'
    )


    dict_quartiers[type].explore(
    m = layers[layer_name], 
    column = 'pav_pour_1000_hab',
    tooltip = ['Quartier', 'population', 'nb_points', 'pav_pour_1000_hab'],
    popup = True,
    cmap = 'Greens',
    legend = False,
    style_kwds = dict(
            fillOpacity = 0.8,
            color = 'green',
            opacity = 0.5),
        highlight_kwds = dict(
            fillOpacity = 0.5)
            )



    map.add_child(layers[layer_name])





# Contrôle de l'affichage des couches
GroupedLayerControl(
    groups={'Type de flux': [layer for layer in layers.values()]},
    collapsed=False,
).add_to(map)



map_title = "Nombre de pav pour 1000 habitants"
title_html = f'<h1 align="center" style="font-size:24px" >{map_title}</h1>'
map.get_root().html.add_child(folium.Element(title_html))


map.save('../cartes/map_densite_pav_pop.html')

